# Boosting

对于Boosting方法来说，有两个问题需要给出答案：

- 第一个是每一轮学习应该如何改变数据的概率分布

- 第二个是如何将各个弱分类器组合起来

- homogeneous weak learners, learns them sequentially in a very adaptative way(a base model depends on the previous ones) and combines them following a deterministic strategy



![image](./0901.png)

ref: https://www.bilibili.com/video/BV1it411q7wy?from=search&seid=16724763162388049402

- D(.)是样本训练集的权重，$\alpha(.)$是弱分类器模型的权重

- 上一期的弱分类器会影响到下一期的弱分类器，不断迭代

## Boosting methods

- Adaboost: boosting + decision tree

- GDBT: gradient boost + decision tree

- xgboost:

## 结合策略（combining）

### 平均法

- 算数平均：对各个弱分类器的预测输出 $\frac{1}{n}\sum{h_i}$

- 加权平均：对各个弱分类器的预测输出 $\frac{1}{n}\sum{w_i h_i}$

### 投票法

- 相对多数投票法： 少数服从多数

- 绝对多数投票法：在相对多数投票法的基础上，要求票过半数

- 加权投票法： 

### 学习法

- stacking: 我们将训练集弱学习器的学习结果作为输入样本，将训练集的输出作为输出，重新训练一个学习器来得到最终结果。

- 在这种情况下，我们将弱学习器称为初级学习器，将用于结合的学习器称为次级学习器。对于测试集，我们首先用初级学习器预测一次，得到次级学习器的输入样本，再用次级学习器预测一次，得到最终的预测结果。

# Adaboost (Adaptive Boosting)

## Step 1: 计算样本权重

- 对训练集中的每一个样本赋予一个权重，构成权重向量D。

- 初始化每个样本权重相等，weight = $\frac{1}{n}$

## Step 2: 训练出一个弱分类器，并计算错误率

error rate  = $\frac{错误的数量}{样本总数}$

## Step 3: 计算弱分类器的权重

- 赋予当前的弱分类器权重$\alpha$


- $\alpha = \frac{1}{2} ln(\frac{1-error\space rate}{error\space rate})$

## Step 4: 调整弱分类器的权重

- 根据上一次训练结果，调整权重值（上一次分对的权重降低，分错的权重增加）


- 如果第i个样本被**正确**分类，则该样本权重更改为：


- 如果第i个样本被**错误**分类，则该样本权重更改为：


## Step 5: 循环以上四步

之后，在同一数据集上再一次训练弱分类器，然后循环上述过程，直到训练错误率为0，或者弱分类器的数目达到指定值。

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline
import seaborn as sns

In [3]:
# 加载训练数据：
wine = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data",header=None)
wine.columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash', 
                'Alcalinity of ash','Magnesium', 'Totalphenols',
                'Flavanoids', 'Nonflavanoid phenols','Proanthocyanins',
                'Color intensity', 'Hue','OD280/OD315 of diluted wines','Proline']

In [4]:
# 数据预处理
# 仅仅考虑2，3类葡萄酒，去除1类
wine = wine[wine['Class label'] != 1]
y = wine['Class label'].values
X = wine[['Alcohol','OD280/OD315 of diluted wines']].values
# 将分类标签变成二进制编码：
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
# 按8：2分割训练集和测试集
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,
                                                 random_state=1,stratify=y) # stratify参数代表了按照y的类别等比例抽样

In [5]:
# 使用单一决策树建模
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(criterion='entropy',random_state=1,max_depth=1)

from sklearn.metrics import accuracy_score

tree = tree.fit(X_train,y_train)
y_train_pred = tree.predict(X_train)
y_test_pred = tree.predict(X_test)
tree_train = accuracy_score(y_train,y_train_pred)
tree_test = accuracy_score(y_test,y_test_pred)

print('Decision tree train/test accuracies %.3f/%.3f' % (tree_train,tree_test))

Decision tree train/test accuracies 0.916/0.875


In [6]:
# 使用sklearn实现Adaboost(基分类器为决策树)
'''
AdaBoostClassifier相关参数：
base_estimator：基本分类器，默认为DecisionTreeClassifier(max_depth=1)
n_estimators：终止迭代的次数
learning_rate：学习率
algorithm：训练的相关算法，{'SAMME'，'SAMME.R'}，默认='SAMME.R'
random_state：随机种子
'''

from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier(base_estimator=tree,n_estimators=500,learning_rate=0.1,random_state=1)
ada = ada.fit(X_train,y_train)
y_train_pred = ada.predict(X_train)
y_test_pred = ada.predict(X_test)
ada_train = accuracy_score(y_train,y_train_pred)
ada_test = accuracy_score(y_test,y_test_pred)
print('Adaboost train/test accuracies %.3f/%.3f' % (ada_train,ada_test))

Adaboost train/test accuracies 1.000/0.917
